In [1]:
import pandas as pd
from pandas import json_normalize
from OTXv2 import OTXv2, IndicatorTypes
from datetime import datetime, timedelta
import requests, json

In [2]:
def all_indicators_df(pulses_df):
    indicators=pd.DataFrame()
    
    for i, r in pulses_df.iterrows():
        ind = json_normalize( otx.get_pulse_indicators(r['id'], limit=10000))
        
        if len(indicators)>0:
            indicators = pd.concat([indicators,ind])
        else:
            indicators = ind
            
    return indicators

In [3]:
def indicators_by_type(pulses_df, unique = False):
    
    ind_types = []
    ind_counts ={}
    ind_ids = {}
    for i, r in pulses_df.iterrows():
        indicators = json_normalize( otx.get_pulse_indicators(r['id'], limit=10000))
        for j, ind in indicators.iterrows():
            #display (ind)
            if ind['type'] in ind_types:
                if not unique:
                    ind_counts [ind['type']] = ind_counts [ind['type']] +1
                else: 
                    ind_ids[ind['type']].add(ind['id'])
            else:
                ind_types.append(ind['type'])
                
                if not unique:
                    ind_counts [ind['type']] = 1
                else: ind_ids[ind['type']] = set([ind['id']])
    if not unique:
        return ind_counts
    else:
        for k,v in ind_ids.items():
            ind_counts[k]=len(v)
        return ind_counts

In [4]:
def filter_indicators_by_type(pulses_df, filter_type):

    filtered_ind = []
    for i, r in pulses_df.iterrows():
        indicators = json_normalize(
             otx.get_pulse_indicators(r['id'], limit=10000))
        for j, ind in indicators.iterrows():
            # display (ind)
            if ind['type'] == filter_type:
                filtered_ind.append(ind)


    return  filtered_ind

In [5]:
def indicators_for_cve(CVE, unique=False):
    cve_pulse_df = json_normalize(otx.search_pulses(CVE)["results"])
    return indicators_by_type(cve_pulse_df, unique = unique)

In [6]:
def get_cve_summary(CVE):
    try:
        CVE_response = requests.get('https://cve.circl.lu/api/cve/'+CVE)
        CVE_summary = json.loads(CVE_response.text)['summary']
        return CVE_summary
    except:
        return ''

In [7]:
def get_ip_locations (ips):
    if len(ips)>100:
        return get_ip_locations (ips[:100])+get_ip_locations (ips[100:])
    
    loc_request = requests.post('http://ip-api.com/batch', data='['+','.join(['"'+ip+'"' for ip in ips])+']')
    return loc_request.json()

In [45]:
from OTXv2 import OTXv2
from OTXv2 import IndicatorTypes
api_key = input()
otx = OTXv2(api_key)

INPUT YOUR OTX KEY HERE


In [10]:
data_90days = json_normalize(otx.getsince((datetime.now() - timedelta(days=90)).isoformat(), limit=10000))
display(len(data_90days))
data_90days.columns

660

Index(['id', 'name', 'description', 'author_name', 'modified', 'created',
       'revision', 'tlp', 'public', 'adversary', 'indicators', 'tags',
       'targeted_countries', 'malware_families', 'attack_ids', 'references',
       'industries', 'extract_source', 'more_indicators'],
      dtype='object')

In [16]:
data_90days['modified_date']=pd.to_datetime(data_90days['modified']).dt.date

In [11]:
indicators = all_indicators_df(data_90days)
len(indicators)

46584

In [18]:
indicators['created_date']=pd.to_datetime(indicators['created']).dt.date

In [21]:
CVE_indicators= indicators.query('type=="CVE"').groupby(['indicator', 'created_date'])['created'].count().reset_index()
CVE_indicators.columns = ['CVE', 'date', 'count']
CVE_indicators.to_json('./CVE_js/CVE_90.json')

In [43]:
IPs = (indicators.query('type=="IPv4"')[['indicator','created_date']])
len(IPs)

236

In [34]:
IP_locations = get_ip_locations(list(IPs['indicator']))
len(IP_locations)

236

In [44]:
IPs = IPs.merge(pd.DataFrame(IP_locations)[['query', 'country']], how='left', left_on = 'indicator', right_on='query')
IPs = IPs.groupby([ 'created_date', 'country'])['indicator'].count().reset_index()
IPs.columns = ['date', 'country', 'count']
IPs.to_json('./CVE_js/IP_90.json')

In [39]:
indicator_types = indicators.groupby(['type', 'created_date'])['created'].count().reset_index()
indicator_types.columns = ['type', 'date', 'count']
indicator_types.to_json('./CVE_js/Types_90.json')

In [40]:
indicator_counts= indicators.groupby(['created_date'])['created'].count().reset_index()
indicator_counts.columns = [ 'date', 'count']
indicator_counts.to_json('./CVE_js/Counts_90.json')

In [41]:
indicator_counts

,date,count
0,2020-06-18,32
1,2020-06-23,86
2,2020-06-24,277
3,2020-07-29,567
4,2020-08-31,35
...,...,...
119,2021-06-18,262
120,2021-06-19,61
121,2021-06-20,66
122,2021-06-21,165
